In [1]:
import torch

import flash
from flash.audio import AudioClassificationData
from flash.image import ImageClassifier

/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:252: UserWarning: Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and will be removed in 0.15. Please access them via the appropriate Weights Enum instead.
  warnings.warn(


In [2]:
#from flash.core.data.utils import download_data

#download_data("https://pl-flash-data.s3.amazonaws.com/urban8k_images.zip", "/workspaces/audio-ml/data/i")


In [3]:
datamodule = AudioClassificationData.from_folders(
    train_folder="/workspaces/audio-ml/data/i/urban8k_images/train",
    val_folder="/workspaces/audio-ml/data/i/urban8k_images/val",
    transform_kwargs=dict(spectrogram_size=(64, 64)),
    batch_size=4,
)

In [4]:
model = ImageClassifier(backbone="resnet18", labels=datamodule.labels)


In [5]:
trainer = flash.Trainer(max_epochs=3, gpus=torch.cuda.device_count())
trainer.finetune(model, datamodule=datamodule, strategy=("freeze_unfreeze", 1))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type           | Params
-------------------------------------------------
0 | train_metrics | ModuleDict     | 0     
1 | val_metrics   | ModuleDict     | 0     
2 | test_metrics  | ModuleDict     | 0     
3 | adapter       | DefaultAdapter | 11.2 M
-------------------------------------------------
14.7 K    Trainable params
11.2 M    Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/site-packages/pytorch_lightning/callbacks/finetuning.py:210: UserWarning: The provided params to be frozen already exist within another group of this optimizer. Those parameters will be skipped.
HINT: Did you init your optimizer in `configure_optimizer` as such:
 <class 'torch.optim.adam.Adam'>(filter(lambda p: p.requires_grad, self.parameters()), ...) 
  rank_zero_warn(


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [7]:
datamodule = AudioClassificationData.from_files(
    predict_files=[
        "/workspaces/audio-ml/data/i/urban8k_images/test/air_conditioner/13230-0-0-5.wav.jpg",
        "/workspaces/audio-ml/data/i/urban8k_images/test/children_playing/9223-2-0-15.wav.jpg",
        "/workspaces/audio-ml/data/i/urban8k_images/test/jackhammer/22883-7-10-0.wav.jpg",
    ],
    batch_size=3,
)
predictions = trainer.predict(model, datamodule=datamodule, output="labels")
print(predictions)


Predicting: 1575it [00:00, ?it/s]

TypeError: new(): invalid data type 'str'

In [8]:
trainer.save_checkpoint("audio_classification_model.pt")
